<a href="https://colab.research.google.com/github/lawgorithm/sql_practice/blob/main/Server_Utilization_Time_%5BAmazon_SQL_Interview_Question%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://datalemur.com/questions/total-utilization-time

In [ ]:
# Website solution, seems wrong based on null handling to me
%%sql
WITH running_time
AS (
  SELECT
    server_id,
    session_status,
    status_time AS start_time,
    LEAD(status_time) OVER (
      PARTITION BY server_id
      ORDER BY status_time) AS stop_time
  FROM server_utilization
)

SELECT
  DATE_PART('days', JUSTIFY_HOURS(SUM(stop_time - start_time))) AS total_uptime_days
FROM running_time
WHERE session_status = 'start'
  AND stop_time IS NOT NULL;

In [ ]:
# Alternative solution that gives the same result
%%sql
WITH running_time
AS (
  SELECT
    server_id,
    session_status,
    status_time AS start_time,
    LEAD(status_time) OVER (
      PARTITION BY server_id
      ORDER BY status_time) AS stop_time
  FROM server_utilization
)

SELECT
  FLOOR(SUM(EXTRACT(EPOCH FROM stop_time) - EXTRACT(EPOCH FROM start_time)) / (60*60*24)) AS total_uptime_days
FROM running_time
WHERE session_status = 'start'
;

In [ ]:
# My solution
%%sql
WITH server_level_data AS (
SELECT
  server_id,
  session_status,
  status_time as stop_time,
  LAG(status_time) OVER (
    PARTITION BY server_id
    ORDER BY status_time ASC
  ) as start_time,
  MAX(status_time) OVER () AS global_max,
  MIN(status_time) OVER () AS global_min
FROM server_utilization
), server_durations AS (
SELECT
  server_id,
  EXTRACT(EPOCH FROM COALESCE(stop_time, global_max)) -
    EXTRACT(EPOCH FROM COALESCE(start_time, global_min)) AS seconds_duration
FROM server_level_data
WHERE
  session_status = 'stop'
)
SELECT
  FLOOR(SUM(seconds_duration) / (60 * 60 * 24))
FROM server_durations